In [1]:
import pandas as pd

In [6]:
#pip install llama-stack

In [5]:
#!pip install transformers accelerate datasets
#!pip install xlsxwriter  # 如果要写 Excel


In [7]:
import pandas as pd

# 修改为你本地实际的路径
csv_path = "/Users/huangchujun/Documents/WCM/Year 1 Summer/Q&A data/untitled folder/cad_questions_combined.csv"
#excel_path_1 = "/Users/huangchujun/Documents/WCM/Year 1 Summer/Q&A data/untitled folder/Dataset_Text.xls"
excel_path_2 = "/Users/huangchujun/Documents/WCM/Year 1 Summer/Q&A data/untitled folder/Dataset_multiple choices.xlsx"

# 读取文件
df_csv = pd.read_csv(csv_path)
#df_excel_1 = pd.read_excel(excel_path_1, engine="xlrd")

df_excel_2 = pd.read_excel(excel_path_2)

# 查看前几行
print("CSV first rows：")
print(df_csv.head())

print("\nExcel first rows：")
print(df_excel_2.head())




CSV first rows：
                                            question  \
0  An ill 16 days old baby girl is brought to the...   
1        Earliest lesion seen in atherosclerosis is:   
2  Flipped LDH indicating Myocardial Infarction i...   
3  A 19 year old female presents with pain in the...   
4  A 63 year old man presents with a triad of ang...   

                                                 exp  cop                 opa  \
0  In CHF pallor, dyspnoea, tachypnoea, tachycard...  1.0                 CHF   
1  Ans: A (Fatty streaks) Ref: Robbins Pathologic...  1.0       Fatty streaks   
2  When LDH-1 is greater than LDH-2, it is referr...  1.0        LDH-1> LDH-2   
3  Answer: c) Aspirin (SCHWARTZ 19TH ED, P-1525; ...  3.0  Atropine injection   
4  Even severe AS may exist for many years withou...  3.0     antral stenosis   

                            opb                 opc  \
0      Glycogen storage disease        Pericarditis   
1            Intimal thickening  Fibrinoid necrosi

In [9]:
#!pip install xlrd
#!pip install openpyxl



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import pandas as pd

# 读取数据
df_csv = pd.read_csv(csv_path)

# 映射关系
choice_map = {
    1.0: "A",
    2.0: "B",
    3.0: "C",
    4.0: "D"
}

# 替换 cop 列的数值
df_csv["cop"] = df_csv["cop"].map(choice_map)

# 查看替换后的结果（可选）
print(df_csv[["cop", "opa", "opb", "opc", "opd"]].head())

# 如果你后续还要用这个修改过的 df_csv 生成训练文本，可以直接继续使用 df_csv


  cop                 opa                           opb                 opc  \
0   A                 CHF      Glycogen storage disease        Pericarditis   
1   A       Fatty streaks            Intimal thickening  Fibrinoid necrosis   
2   A        LDH-1> LDH-2                 LDH-2 > LDH-1       LDH-4 > LDH-5   
3   C  Atropine injection  Levothyroxine administration             Aspirin   
4   C     antral stenosis       Tricuspid regurgitation       Aoic stenosis   

                                                 opd  
0  Aberrant left coronary aery arising from pulmo...  
1                                             Plaque  
2                                      LDH-5 > LDH-4  
3                     Increase iodine intake in food  
4                                 Aoic regurgitation  


In [9]:
import pandas as pd

# 读取你的数据
#df_csv = pd.read_csv("你的CSV路径")
df_excel = pd.read_excel(excel_path_2)

# 示例处理 CSV 数据
csv_prompts = []
for _, row in df_csv.iterrows():
    if row["choice_type"] == "single":
        question = row["question"]
        options = f"A. {row['opa']}\nB. {row['opb']}\nC. {row['opc']}\nD. {row['opd']}"
        answer = row["cop"]
        prompt = f"### Instruction:\n{question}\n\n{options}\n\n### Response:\n{answer}"
        csv_prompts.append(prompt)

# 示例处理 Excel 数据
excel_prompts = []
for _, row in df_excel.iterrows():
    question = row["Question"]
    options = f"A. {row['Option A']}\nB. {row['Option B']}\nC. {row['Option C']}\nD. {row['Option D']}"
    answer = row["Answer"]
    prompt = f"### Instruction:\n{question}\n\n{options}\n\n### Response:\n{answer}"
    excel_prompts.append(prompt)

# 合并所有 prompts 并保存
all_prompts = csv_prompts + excel_prompts
with open("train_data.txt", "w", encoding="utf-8") as f:
    for item in all_prompts:
        f.write(item + "\n\n")


In [10]:
# 打印前5个 CSV prompts
print("CSV Prompts:")
for prompt in csv_prompts[:5]:
    print(prompt)
    print("=" * 50)

# 打印前5个 Excel prompts
print("Excel Prompts:")
for prompt in excel_prompts[:5]:
    print(prompt)
    print("=" * 50)


CSV Prompts:
### Instruction:
Earliest lesion seen in atherosclerosis is:

A. Fatty streaks
B. Intimal thickening
C. Fibrinoid necrosis
D. Plaque

### Response:
A
### Instruction:
Flipped LDH indicating Myocardial Infarction is represented by:

A. LDH-1> LDH-2
B. LDH-2 > LDH-1
C. LDH-4 > LDH-5
D. LDH-5 > LDH-4

### Response:
A
### Instruction:
A 63 year old man presents with a triad of angina, syncope and congestive hea failure. Which of the following valvular hea lesion can be suspected-

A. antral stenosis
B. Tricuspid regurgitation
C. Aoic stenosis
D. Aoic regurgitation

### Response:
C
### Instruction:
In a patient with a type of familial dyslipidemias he presented with palmar Xanthomas and he ias at increased risk of atherosclerosis and CAD.In Lipid profile there was elevation of triacylglycerols and cholesterol.And the IDL and chylomicrons were elevated.What is the pathophysiology of the condition?

A. LDL deficiency
B. VLDL overproduction
C. Apo C-2 deficiency
D. Apo E deficienc

In [2]:
import requests
import json

# ✏️ 你部署的 LLaMA 模型接口地址
url = "http://localhost:5001/llama"


# ✅ 构造系统提示和问题
system_message = "You are a helpful medical assistant."
user_message = """What is the earliest lesion in atherosclerosis?
A. Fatty streaks
B. Intimal thickening
C. Fibrinoid necrosis
D. Plaque"""

# ✅ 构造 prompt（llama.cpp 格式）
prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

# ✅ 发请求（max_tokens 可按需调整）
response = requests.post(
    url,
    json={
        "system_message": system_message,
        "user_message": user_message,
        "max_tokens": 50
    }
)

# ✅ 解析模型响应
result = response.json()
print("Model output:\n")
print(result["choices"][0]["text"])



Model output:

<s>[INST] <<SYS>>
You are a helpful medical assistant.
<</SYS>>
What is the earliest lesion in atherosclerosis?
A. Fatty streaks
B. Intimal thickening
C. Fibrinoid necrosis
D. Plaque [/INST]  As a helpful medical assistant, I'm glad you asked! The earliest lesion in atherosclerosis is:
D. Plaque.
Atherosclerosis is a chronic inflammatory condition that


In [11]:
import requests

# ✅ 模型部署地址
url = "http://localhost:5001/llama"
system_message = "You are a helpful medical assistant."

# ✅ 用你生成的 prompts（去掉答案部分）
for prompt in excel_prompts[:10]:  # ← 只跑前10题可以调试，后续改成全跑

    # 分离 prompt 和答案部分
    if "### Response:" in prompt:
        instruction = prompt.split("### Response:")[0].replace("### Instruction:", "").strip()
    else:
        continue  # 如果格式不对，跳过

    # 发请求给模型
    response = requests.post(
        url,
        json={
            "system_message": system_message,
            "user_message": instruction,
            "max_tokens": 50
        }
    )

    # 输出模型回复
    try:
        result = response.json()
        answer = result["choices"][0]["text"].strip()
        print(f"Q: {instruction}\n🧠 Model Answer: {answer}\n{'-'*40}")
    except Exception as e:
        print("Error:", e)


Q: Which of the following describes the cluster of blood capillaries found in each nephron in the kidney?

A. Afferent arteriole
B. Glomerulus
C. Loop of Henle
D. Renal pelvis
🧠 Model Answer: <s>[INST] <<SYS>>
You are a helpful medical assistant.
<</SYS>>
Which of the following describes the cluster of blood capillaries found in each nephron in the kidney?

A. Afferent arteriole
B. Glomerulus
C. Loop of Henle
D. Renal pelvis [/INST]  Great, I'm happy to help! The correct answer is (B) Glomerulus.
The glomerulus is the cluster of blood capillaries found in each nephron in the kidney. It is a network of
----------------------------------------
Q: Oxygenated blood is carried to the heart by which of the following structures?

A. Aorta
B. Carotid arteries
C. Inferior vena cava
D. Pulmonary veins
🧠 Model Answer: <s>[INST] <<SYS>>
You are a helpful medical assistant.
<</SYS>>
Oxygenated blood is carried to the heart by which of the following structures?

A. Aorta
B. Carotid arteries
C. Infer